In [1]:
import numpy as np
import pandas as pd
import folium

from src.dbtool import get_eved_db
from folium.vector_layers import PolyLine, CircleMarker

In [2]:
def fit_bounding_box(html_map, bb_list):
    if isinstance(bb_list, list):
        ll = np.array(bb_list)
    else:
        ll = bb_list

    min_lat, max_lat = ll[:, 0].min(), ll[:, 0].max()
    min_lon, max_lon = ll[:, 1].min(), ll[:, 1].max()
    html_map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    return html_map

In [3]:
def create_map() -> folium.Map:
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=21, control_scale=True)
    return html_map

In [4]:
def show_trajectory(html_map: folium.Map, 
                    trajectory: np.ndarray, 
                    color: str = "red",
                    radius: int = 1) -> folium.Map:
    for lat, lon in trajectory:
        CircleMarker(location=[lat, lon], radius=radius, color=color, fill=True, fill_color=color).add_to(html_map)
    return html_map

In [5]:
db = get_eved_db()

In [14]:
df = db.load_dr_trajectory(100).ffill()

In [15]:
df

,signal_id,match_latitude,match_longitude,t,latitude,longitude,speed,accel,jerk
0,11752228,42.295429,-83.705258,0.0,42.295429,-83.705258,27.398112,8.903264e-01,1.298628e+00
1,11752229,42.295429,-83.705258,1.5,42.295344,-83.704772,28.733602,2.838268e+00,1.400125e+00
2,11752230,42.295429,-83.705258,1.7,42.295315,-83.704713,29.353201,3.121000e+00,6.461304e-01
3,11752231,42.295429,-83.705258,3.6,42.294746,-83.704791,35.698361,-9.505235e+00,-7.431610e+00
4,11752232,42.294667,-83.704868,3.9,42.294667,-83.704868,32.238353,-1.177196e+01,-4.997606e+00
...,...,...,...,...,...,...,...,...,...
550,11752778,42.257636,-83.700053,444.5,42.257626,-83.700004,0.000000,3.315866e-12,1.657933e-11
551,11752779,42.257636,-83.700053,444.5,42.257626,-83.700004,0.000000,3.315866e-12,1.657933e-11
552,11752780,42.257636,-83.700053,444.5,42.257626,-83.700004,0.000000,3.315866e-12,1.657933e-11
553,11752781,42.257636,-83.700053,444.5,42.257626,-83.700004,0.000000,3.315866e-12,1.657933e-11


In [16]:
unique_df = df.drop_duplicates(subset=["match_latitude", "match_longitude"], keep="first")

In [17]:
m = create_map()
m = fit_bounding_box(m, unique_df[["match_latitude", "match_longitude"]].to_numpy())
m = show_trajectory(m, df[["latitude", "longitude"]].to_numpy(), color="blue")
m = show_trajectory(m, unique_df[["match_latitude", "match_longitude"]].to_numpy())
m